# Save MLFlow model

### Run MNIST CNN training

In [1]:
!pip install keras --user

In [2]:
import mlflow.keras
mlflow.set_tracking_uri("http://192.168.0.243:5005/")

In [3]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import os
import time

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:

# # Configure output folder to store TF events
output_root = "/mnt/tardis6/hvd-demo/logs/"
output_dir = "/dbfs" + output_root
os.environ['OUTPUT_DIR'] = output_dir

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
# mlflow.create_experiment("mnist-test")

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'1'

In [6]:
%load_ext tensorboard

In [7]:
import tensorflow as tf
import datetime

In [8]:
# Clear any logs from previous runs
# rm -rf ./logs/

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [11]:

with mlflow.start_run(experiment_id = 1) as run:
    # Record Start time
    start_time = time.time()
    # Configure for TensorBoard visualization
    #   Reference: [Monitor progress of your Keras based neural network using Tensorboard](https://bit.ly/2C36EBJ)
    # tensorboard = TensorBoard(log_dir=output_dir)
        
    batch_size = 128
    num_classes = 10
    epochs = 12

    # input image dimensions
    img_rows, img_cols = 28, 28

    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test)
             )
              # ,callbacks=[tensorboard_callback])
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
        # Record Complete Time and determine Elapsed Time
    complete_time = time.time()
    elapsed_time = complete_time - start_time

    # Log MLflow Parameters
    mlflow.log_param("mode", "single node")
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("num_classes", num_classes)
    
    # Log MLflow Metrics
    mlflow.log_metric("Test Loss", score[0])
    mlflow.log_metric("Test Accuracy", score[1])
    mlflow.log_metric("Duration", elapsed_time)
    
    # Log Model
#     mlflow.keras.log_model(model, "models")
    mlflow.keras.log_model(
        keras_model=model,
        artifact_path="mnist-model",
        registered_model_name="mnist-model"
    )

    # Log TF events to MLflow
    mlflow.log_artifacts(output_dir, artifact_path="tf.events")
      
    # Return RunUUID
    #return run.info

# Close TF session
sess.close()

# Complete MLflow Run
mlflow.end_run() 

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/12
469/469 [==============================] - 87s 185ms/step - loss: 2.3004 - accuracy: 0.1135 - val_loss: 2.2756 - val_accuracy: 0.2237
Epoch 2/12
469/469 [==============================] - 91s 194ms/step - loss: 2.2592 - accuracy: 0.2063 - val_loss: 2.2264 - val_accuracy: 0.3779
Epoch 3/12
469/469 [==============================] - 94s 200ms/step - loss: 2.2071 - accuracy: 0.3037 - val_loss: 2.1584 - val_accuracy: 0.5078
Epoch 4/12
469/469 [==============================] - 99s 212ms/step - loss: 2.1337 - accuracy: 0.3992 - val_loss: 2.0637 - val_accuracy: 0.5957
Epoch 5/12
469/469 [==============================] - 100s 214ms/step - loss: 2.0324 - accuracy: 0.4745 - val_loss: 1.9328 - val_accuracy: 0.6654
Epoch 6/12
469/469 [==============================] - 92s 196ms/step - loss: 1.8936 - accuracy: 0.5378 - val_loss: 1.7575 - val_accuracy: 0.7262
Epoch 7/12
469/469 [==============================] - 93

PermissionError: [Errno 13] Permission denied: '/home/mlflow'

In [ ]:
%tensorboard --logdir logs/fit

### Save model to files

In [3]:
!pip install mlflow --user

In [5]:
import mlflow.keras

path = "./model"
mlflow.keras.save_model(model, path)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


MlflowException: Path '/home/jovyan/notebooks/mlflow-keras-mnist/model' already exists

In [9]:

path = "./mnist-model"

mlflow.keras.save_model(model, path)


# import mlflow
# remote_server_uri = "http://172.31.160.1:5005/" # set to your server URI
# mlflow.set_tracking_uri(remote_server_uri)
# # Note: on Databricks, the experiment name passed to mlflow_set_experiment must be a
# # valid path in the workspace
# mlflow.set_experiment("/my-experiment")

MlflowException: Path '/home/jovyan/notebooks/mlflow-keras-mnist/mnist-model' already exists